1. Überprüfen ob "Description" und "Text" der Artikel deckungsgleich sind und gegebenenfalls die betroffenen Artikel entfernen.
2. Nicht-lateinische Schrift entfernen und prüfen ob Heuristiken zur gezielten Entfernung von nicht-deutscher Sprache gefunden werden können (z.B. Originaltitel von Filmen in Klammern hinter dem Artikelnamen)
3. Mindestlänge von Artikeln auf 100 Wörter und Maximallänge auf 2000 Wörter festlegen. Artikel die weniger als 100 Wörter enthalten, werden entfernt und durch längere ersetzt. Artikel mit mehr als 2000 Wörtern werden an der Obergrenze abgeschnitten.
4. Überlappungen zwischen den verschiedenen Datensätzen vor dem Merge prüfen.

In [1]:
import pandas as pd
import regex

%matplotlib inline

In [2]:
categories = pd.read_csv("../data/kategorien.csv")
categories.head(5)

,category,description,text
0,Herrscher,"Adaloald (auch Adalwald, Adulubaldus; * 602 in...","Adaloald (auch Adalwald, Adulubaldus; * 602 in..."
1,Herrscher,"Ælfwine (auch Aelfwine, Elfwine, Aelfuini, Ael...","Ælfwine (auch Aelfwine, Elfwine, Aelfuini, Ael..."
2,Herrscher,Æscwine (auch: Aescwine; Escuuine) († 676) war...,Æscwine (auch: Aescwine; Escuuine) († 676) war...
3,Herrscher,"Æthelberht I. (auch Æþelbryht, Æþelbyrht, Aedi...","Æthelberht I. (auch Æþelbryht, Æþelbyrht, Aedi..."
4,Herrscher,"Æthelburg (auch Æthelburh, Ethelburga, Æthelbu...","Æthelburg (auch Æthelburh, Ethelburga, Æthelbu..."


### 1. description == text

In [3]:
categories = categories[categories["description"] != categories["text"]].reset_index(drop=True)

### 2. Nicht-lateinische Schrift

In [4]:
token_pattern = regex.compile(r"\p{Latin}+\p{Connector_Punctuation}?\p{Latin}+|\p{Number}+")

In [5]:
example = """Deutsch: lateinisches Alphabet, 
             Amharisch: የላቲን አልፋቤት,
             Bengalisch: লাতিন বর্ণমালা,
             Arabisch: اَللُّغَةُ اَلْعَرَبِيَّة,
             Chinesisch: 漢字,
             Japanisch: 日本語"""
token_pattern.findall(example)

['Deutsch',
 'lateinisches',
 'Alphabet',
 'Amharisch',
 'Bengalisch',
 'Arabisch',
 'Chinesisch',
 'Japanisch']

In [6]:
def drop_nonlatin(articles):
    return articles.apply(lambda article: " ".join(token_pattern.findall(str(article))))

In [7]:
categories["description"] = drop_nonlatin(categories["description"])
categories["text"] = drop_nonlatin(categories["text"])

### 3. Mindest- bzw. Maximallänge

In [8]:
minimum = 100
maximum = 2000

In [9]:
length = list()

for article in categories["text"]:
    tokens = token_pattern.findall(article)
    length.append(len(tokens))

categories["length"] = length

In [10]:
categories.head(5)

,category,description,text,length
0,Herrscher,Adaloald auch Adalwald Adulubaldus 602 in Monz...,Adaloald auch Adalwald Adulubaldus 602 in Monz...,236
1,Herrscher,Ælfwine auch Aelfwine Elfwine Aelfuini Aelfwin...,Ælfwine auch Aelfwine Elfwine Aelfuini Aelfwin...,279
2,Herrscher,Æscwine auch Aescwine Escuuine 676 war von 674...,Æscwine auch Aescwine Escuuine 676 war von 674...,135
3,Herrscher,Æthelberht auch Æþelbryht Æþelbyrht Aedilberct...,Æthelberht auch Æþelbryht Æþelbyrht Aedilberct...,841
4,Herrscher,Æthelburg auch Æthelburh Ethelburga Æthelburga...,Æthelburg auch Æthelburh Ethelburga Æthelburga...,303


In [11]:
categories = categories[categories["length"] >= minimum].reset_index(drop=True)

In [12]:
cut = categories[categories["length"] > maximum]

In [13]:
for row, article in cut.iterrows():
    tokens = token_pattern.findall(article["text"])
    categories.iloc[row, 2] = " ".join(tokens[:maximum])

In [14]:
length = list()

for article in categories["text"]:
    tokens = token_pattern.findall(article)
    length.append(len(tokens))

categories["length"] = length

In [15]:
categories = categories.drop_duplicates("description")

In [16]:
categories["category"].value_counts()

Soziologie        237
Krankheit         232
Computerspiele    200
Kriege            200
Burgen            200
Herrscher         200
Film              200
Musik             200
Fernsehserien     200
Literatur         200
Name: category, dtype: int64